<a href="https://colab.research.google.com/github/FerbeiPatricia/PMP-2024/blob/main/Lab3_PMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install pgmpy

In [9]:
# Instalare pachet pgmpy, daca nu este deja instalat
# !pip install pgmpy

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. Definirea structurii rețelei bayesiene
model = BayesianNetwork([
    ('S', 'O'),  # S influențează O
    ('S', 'L'),  # S influențează L
    ('S', 'M'),  # S influențează M
    ('L', 'M')   # L influențează M
])

# 2. Definirea tabelelor de probabilități condiționate (CPD)

# CPD pentru Spam (S)
cpd_s = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]], state_names={'S': [0, 1]})

# CPD pentru O (conținerea cuvântului "ofertă") condiționată de S
cpd_o = TabularCPD(variable='O', variable_card=2,
                   values=[[0.9, 0.3],  # P(O=0 | S=0), P(O=0 | S=1)
                           [0.1, 0.7]], # P(O=1 | S=0), P(O=1 | S=1)
                   evidence=['S'], evidence_card=[2],
                   state_names={'O': [0, 1], 'S': [0, 1]})

# CPD pentru L (conținerea link-urilor) condiționată de S
cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.7, 0.2],  # P(L=0 | S=0), P(L=0 | S=1)
                           [0.3, 0.8]], # P(L=1 | S=0), P(L=1 | S=1)
                   evidence=['S'], evidence_card=[2],
                   state_names={'L': [0, 1], 'S': [0, 1]})

# CPD pentru M (lungimea e-mailului) condiționată de S și L
cpd_m = TabularCPD(variable='M', variable_card=2,
                   values=[[0.8, 0.4, 0.2, 0.1],  # P(M=0 | S=0, L=0), P(M=0 | S=0, L=1), ...
                           [0.2, 0.6, 0.8, 0.9]], # P(M=1 | S=0, L=0), P(M=1 | S=0, L=1), ...
                   evidence=['S', 'L'], evidence_card=[2, 2],
                   state_names={'M': [0, 1], 'S': [0, 1], 'L': [0, 1]})

# Adăugarea CPD-urilor în model
model.add_cpds(cpd_s, cpd_o, cpd_l, cpd_m)

# 3. Verificarea modelului
assert model.check_model()

# 4. Realizarea inferenței
inference = VariableElimination(model)

# a) Verificăm independența condiționată
# P(O=1 | S=1) - probabilitatea ca e-mailul să conțină "ofertă" dat fiind că este spam
prob_O_given_S = inference.query(variables=['O'], evidence={'S': 1})
print("P(O=1 | S=1):")
print(prob_O_given_S)

# P(L=1 | S=1) - probabilitatea ca e-mailul să conțină link-uri dat fiind că este spam
prob_L_given_S = inference.query(variables=['L'], evidence={'S': 1})
print("\nP(L=1 | S=1):")
print(prob_L_given_S)

# Verificăm dacă O este independentă de L condiționat de S:
# Calculăm P(O=1, L=1 | S=1)
joint_prob = inference.query(variables=['O', 'L'], evidence={'S': 1})
print("\nP(O=1, L=1 | S=1):")
print(joint_prob)

# P(L=1 | S=1) - pentru a verifica independența
prob_L_given_S1 = inference.query(variables=['L'], evidence={'S': 1, 'O': 1})
print("\nP(L=1 | S=1, O=1):")
print(prob_L_given_S1)

# P(O=1 | S=1, L=1) - pentru a verifica independența
prob_O_given_S1_L1 = inference.query(variables=['O'], evidence={'S': 1, 'L': 1})
print("\nP(O=1 | S=1, L=1):")
print(prob_O_given_S1_L1)


P(O=1 | S=1):
+------+----------+
| O    |   phi(O) |
+======+==========+
| O(0) |   0.3000 |
+------+----------+
| O(1) |   0.7000 |
+------+----------+

P(L=1 | S=1):
+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.2000 |
+------+----------+
| L(1) |   0.8000 |
+------+----------+

P(O=1, L=1 | S=1):
+------+------+------------+
| O    | L    |   phi(O,L) |
+======+======+============+
| O(0) | L(0) |     0.0600 |
+------+------+------------+
| O(0) | L(1) |     0.2400 |
+------+------+------------+
| O(1) | L(0) |     0.1400 |
+------+------+------------+
| O(1) | L(1) |     0.5600 |
+------+------+------------+

P(L=1 | S=1, O=1):
+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.2000 |
+------+----------+
| L(1) |   0.8000 |
+------+----------+

P(O=1 | S=1, L=1):
+------+----------+
| O    |   phi(O) |
+======+==========+
| O(0) |   0.3000 |
+------+----------+
| O(1) |   0.7000 |
+------+----------+


In [10]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Definirea structurii rețelei bayesiene
model = BayesianNetwork([
    ('S', 'O'),  # S influențează O
    ('S', 'L'),  # S influențează L
    ('S', 'M'),  # S influențează M
    ('L', 'M')   # L influențează M
])

# Definirea tabelelor de probabilitate condiționată (CPD)
cpd_s = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]], state_names={'S': [0, 1]})

cpd_o = TabularCPD(variable='O', variable_card=2,
                   values=[[0.9, 0.3],  # P(O=0 | S=0), P(O=0 | S=1)
                           [0.1, 0.7]], # P(O=1 | S=0), P(O=1 | S=1)
                   evidence=['S'], evidence_card=[2],
                   state_names={'O': [0, 1], 'S': [0, 1]})

cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.7, 0.2],  # P(L=0 | S=0), P(L=0 | S=1)
                           [0.3, 0.8]], # P(L=1 | S=0), P(L=1 | S=1)
                   evidence=['S'], evidence_card=[2],
                   state_names={'L': [0, 1], 'S': [0, 1]})

cpd_m = TabularCPD(variable='M', variable_card=2,
                   values=[[0.8, 0.4, 0.2, 0.1],  # P(M=0 | S=0, L=0), P(M=0 | S=0, L=1), ...
                           [0.2, 0.6, 0.8, 0.9]], # P(M=1 | S=0, L=0), P(M=1 | S=0, L=1), ...
                   evidence=['S', 'L'], evidence_card=[2, 2],
                   state_names={'M': [0, 1], 'S': [0, 1], 'L': [0, 1]})

# Adăugarea CPD-urilor în model
model.add_cpds(cpd_s, cpd_o, cpd_l, cpd_m)

# Verificarea modelului
assert model.check_model()

# Realizarea inferenței
inference = VariableElimination(model)

# Clasificarea e-mailurilor în funcție de atributele O, L și M
# Cazuri de testare
test_cases = [
    {'O': 1, 'L': 1},  # E-mail cu "ofertă" și link
    {'O': 1, 'L': 0},  # E-mail cu "ofertă" și fără link
    {'O': 0, 'L': 1},  # E-mail fără "ofertă" și cu link
    {'O': 0, 'L': 0}   # E-mail fără "ofertă" și fără link
]

# Evaluarea probabilităților de spam în funcție de atributele testate
for case in test_cases:
    # Obținerea probabilităților condiționate pentru spam
    prob_spam = inference.query(variables=['S'], evidence=case)
    print(f'Probabilitățile pentru cazul O={case["O"]}, L={case["L"]}:')
    print(prob_spam)


Probabilitățile pentru cazul O=1, L=1:
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.0744 |
+------+----------+
| S(1) |   0.9256 |
+------+----------+
Probabilitățile pentru cazul O=1, L=0:
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.4286 |
+------+----------+
| S(1) |   0.5714 |
+------+----------+
Probabilitățile pentru cazul O=0, L=1:
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.6279 |
+------+----------+
| S(1) |   0.3721 |
+------+----------+
Probabilitățile pentru cazul O=0, L=0:
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.9403 |
+------+----------+
| S(1) |   0.0597 |
+------+----------+
